# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

def make_df(cur):
    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    return df

In [ ]:
# Your code here; create the connection and cursor

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [14]:
# Your code here; use a subquery. No join is necessary 
df = make_df(cur.execute('''SELECT customerNumber, contactLastName, contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT customerNumber
                                FROM orders
                                WHERE orderDate = '2003-01-31');'''))
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [21]:
# Your code here; use a subquery. No join is necessary 
df = make_df(cur.execute('''SELECT productCode, productName, count(productCode) AS orderNumber, SUM(quantityOrdered) AS totalSold
                            FROM products p
                            JOIN orderDetails
                            USING(productCode)
                            GROUP BY productName
                            ORDER BY totalSold DESC;'''))
df

,productCode,productName,orderNumber,totalSold
0,S18_3232,1992 Ferrari 360 Spider red,53,1808
1,S18_1342,1937 Lincoln Berline,28,1111
2,S700_4002,American Airlines: MD-11S,28,1085
3,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,S50_1341,1930 Buick Marquette Phaeton,28,1074
...,...,...,...,...
104,S18_2870,1999 Indy 500 Monte Carlo SS,25,855
105,S18_2248,1911 Ford Town Car,25,832
106,S24_3969,1936 Mercedes Benz 500k Roadster,25,824
107,S24_1046,1970 Chevy Chevelle SS 454,25,803


In [26]:
cur.execute("""SELECT productName, COUNT(orderNumber) AS numberOrders, SUM(quantityOrdered) as totalUnitsSold
               FROM products
               JOIN orderdetails
               USING (productCode)
               GROUP BY productName
               ORDER BY totalUnitsSold desc;
               """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.shape)
df.tail()

(109, 3)


,productName,numberOrders,totalUnitsSold
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803
108,1957 Ford Thunderbird,24,767


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [30]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute("""SELECT orderNumber, COUNT(orderNumber) as numberOrders, 
               count(DISTINCT customerNumber) as numberPurchasers,
               SUM(quantityOrdered) as totalUnitsSold
               FROM products
               JOIN orderdetails
               USING (productCode)
               JOIN orders
               USING (orderNumber)
               GROUP BY productName
               ORDER BY totalUnitsSold desc;
               """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(df.shape)
df

(109, 4)


,orderNumber,numberOrders,numberPurchasers,totalUnitsSold
0,10104,53,40,1808
1,10102,28,25,1111
2,10106,28,26,1085
3,10106,28,26,1076
4,10106,28,23,1074
...,...,...,...,...
104,10109,25,21,855
105,10100,25,23,832
106,10100,25,21,824
107,10110,25,20,803


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [43]:
# Your code here:
df = make_df(cur.execute('''SELECT DISTINCT employeeNumber, firstName, lastName, o.city, o.officeCode
                            FROM employees e
                            JOIN offices o 
                            USING(officeCode)
                            JOIN customers c
                            ON e.employeeNumber = c.salesRepEmployeeNumber
                            JOIN orders o
                            USING (customerNumber)
                            JOIN orderDetails
                            USING(orderNumber)
                            WHERE productCode IN (SELECT productCode
                                                   FROM products
                                                   JOIN orderdetails
                                                   USING (productCode)
                                                   JOIN orders
                                                   USING (orderNumber)
                                                   GROUP BY productCode
                                                   HAVING count(DISTINCT customerNumber) < 20);'''))
df

,employeeNumber,firstName,lastName,city,officeCode
0,1370,Gerard,Hernandez,Paris,4
1,1501,Larry,Bott,London,7
2,1337,Loui,Bondur,Paris,4
3,1166,Leslie,Thompson,San Francisco,1
4,1286,Foon Yue,Tseng,NYC,3
5,1612,Peter,Marsh,Sydney,6
6,1611,Andy,Fixter,Sydney,6
7,1401,Pamela,Castillo,Paris,4
8,1621,Mami,Nishi,Tokyo,5
9,1323,George,Vanauf,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [51]:
# Your code here
df = make_df(cur.execute('''SELECT employeeNumber, firstName, lastName, 
                            COUNT(DISTINCT customerNumber) as number_of_customers,
                            AVG(creditLimit)
                            FROM employees e
                            JOIN customers c
                            ON e.employeeNumber = c.salesRepEmployeeNumber
                            GROUP BY employeeNumber
                            
                            '''))
display(df)

,employeeNumber,firstName,lastName,number_of_customers,AVG(creditLimit)
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000


In [53]:
# Your code here
df = make_df(cur.execute('''SELECT AVG(creditLimit)
                            FROM customers
                            
                            '''))
display(df)

,AVG(creditLimit)
0,67659.016393


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!